## GODEEEP LDV Electricification Load Downscaling

Converts GCAM output to 8760 timeseries per Balancing Authority (BA) of Light Duty Vehicle (LDV) transportation load in the Western Electricity Coordinating Council (WECC)

Data required:
* GCAM assumptions input & GCAM simulation output
  * The processed files are included as part of this repository in the input folder
* GCAM mapping between assumptions size.classes and output subsectors
  * This file is redistributed as part of this repository: [input/UCD_techs_revised.csv](../input/UCD_techs_revised.csv).
* Electric Vehicle (EV) distribution by county; for instance in the appendix of:
  * M. Kintner-Meyer, S. Davis, S. Sridhar, D. Bhatnagar, S. Mahserejian and M. Ghosal, "Electric vehicles at scale-phase I analysis: High EV adoption impacts on the western US power grid", Tech. Rep., 2020.
  * This dataset should be augmented with the state abbreviations, county FIPS codes, and Balancing Authority
  * The county to balancing authority mapping should match that provided by TELL, redistributed as part of this repository: [input/ba_service_territory_2019.csv](../input/ba_service_territory_2019.csv).
  * This augmented file is redistributed as part of this repository: [input/EV_at_scale_2020_Kintner-Meyer_etal.csv](../input/EV_at_scale_2020_Kintner-Meyer_etal.csv).
* County mean meteorology for desired year and a day before and after; this is the same data produced as input to Tell
  * Data derived from [TGW Simulations](https://tgw-data.msdlive.org/) available [here](https://doi.org/10.57931/1960548).
* County timezones
  * weather.gov provides the expected mapping here: https://www.weather.gov/gis/ZoneCounty
  * Time Zones: Time zone assignments use the following codes:
    * V   Atlantic Standard
    * E   Eastern Standard
    * C   Central Standard
    * M   Mountain Standard (m = daylight time not observed)
    * P   Pacific Standard
    * A   Alaska Standard
    * H   Hawaii-Aleutian Standard (h = daylight time observed)
    * G   Guam & Marianas
    * J   Japan Time
    * S   Samoa Standard
  * Two letters appear for the nine (9) counties (10 records total) which are divided by a time zone boundary, which are located in the states of FL (Gulf), ID (Idaho), ND (McKenzie, Dunn, and Sioux), NE (Cherry), OR (Malheur), SD (Stanley), and TX (Culberson).
  * For simplification, we currently assume Daylight Savings Time is always active, except where it is not currently honored at all


<br/>

<br/>

In [ ]:
import gcamreader
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import pickle
plt.rcParams['figure.figsize'] = [20, 10]
plt.rcParams['figure.dpi'] = 100

#### Get transportation assumptions and outputs from the GODEEEP version of GCAM-USA
You can skip these steps if using the pre-processed files

In [ ]:
# GCAM transportation assumptions files; prefer values in first file
GCAM_path = '.'
transportation_assumption_files = [
    f'{GCAM_path}/input/gcamdata/inst/extdata/energy/OTAQ_trn_data_EMF37.csv',
    f'{GCAM_path}/input/gcamdata/inst/extdata/energy/UCD_trn_data_CORE.csv',
]

In [ ]:
transportation_assumptions = [pd.read_csv(f, comment='#') for f in transportation_assumption_files]

In [ ]:
# merge both files but prefer values in the first file (based on guidance from Yang)
merged = transportation_assumptions[0][transportation_assumptions[0].UCD_region == 'USA'].merge(transportation_assumptions[1][transportation_assumptions[1].UCD_region == 'USA'], how='outer', on=['UCD_region', 'UCD_sector', 'mode', 'size.class', 'UCD_technology', 'UCD_fuel', 'variable'], suffixes=('_primary', '_secondary'))
primary = merged[[c for c in merged.columns if not '_secondary' in c]]
primary.columns = primary.columns.str.replace('_primary', '')
secondary = merged[[c for c in merged.columns if not '_primary' in c]]
secondary.columns = secondary.columns.str.replace('_secondary', '')
assumptions = primary.combine_first(secondary)
# write assumptions to excel file
assumptions.to_excel('../input/transportation_assumptions_godeeep.xlsx', index=False)

In [ ]:
# get the service output from GCAM
for gcam_scenario in ['BAU_Climate', 'NetZeroNoCCS_Climate']:
    gcam_db_path = '.'
    gcam_db_file = f'Main_database_basexdb_{gcam_scenario}'
    query_path = '../input/queries.xml'
    conn = gcamreader.LocalDBConn(gcam_db_path, gcam_db_file)
    queries = gcamreader.parse_batch_query(query_path)
    service_output = conn.runQuery(queries[0])
    energy_output = conn.runQuery(queries[1])
    # write to excel files
    service_output.to_csv(f'../input/transportation_service_output_godeeep_{gcam_scenario}.csv', index=False)
    energy_output.to_csv(f'../input/transportation_energy_output_godeeep_{gcam_scenario}.csv', index=False)

#### Generate LDV load profiles
This could be restructured to loop over all scenarios and years, but for now you will need to manually toggle your desired settings

In [ ]:
# establish the weather year to use, weather scenario, and gcam scenario
year = 2030
weather_scenario = 'rcp85hotter'
# weather_scenario = 'rcp45cooler'
# gcam_scenario = 'BAU_Climate'
gcam_scenario = 'NetZeroNoCCS_Climate'

In [ ]:
# charging parameters to use
pev_dist='BEV'
class_dist='Equal'
pref_dist='Home60'
home_power_dist='MostL2'
work_power_dist='MostL2'

home_access_dist='HA75'
# home_access_dist='HA100'
res_charging='min_delay'
# res_charging='load_leveling'
work_charging='min_delay'
# work_charging='load_leveling'

In [ ]:
# get fleet size and daily miles breakdown by county
fleet_size = LDV.get_fleet_by_county(
    f'../input/transportation_service_output_godeeep_{gcam_scenario}.csv',
    '../input/transportation_assumptions_godeeep.xlsx',
    '../input/UCD_techs_revised.csv',
    '../input/EV_at_scale_2020_Kintner-Meyer_etal.csv',
    '../input/county_timezones.dbx',
    year,
)

In [ ]:
# sanity check
fleet_size[fleet_size.FIPS==53033], fleet_size[fleet_size.FIPS==53033].sum()

In [ ]:
# Get all the possible temperature load profiles for each county
# This will take awhile if going for all of WECC (~4+ hours)
# Consider using the `county_subset` option for a smaller test run

# You will need to provide your own `./input/nrel-api-key` file after signing up for the NREL API service

temperature_load_profiles = LDV.get_temperature_loads_by_county(
    fleet_size,
    'https://developer.nrel.gov/api/evi-pro-lite/v1/daily-load-profile',
    '../input/nrel-api-key',
    #balancing_authority_subset=['IID'],
    state_subset=['CA','OR','WA','AZ','NV','WY','ID','UT','NM','CO','MT'],
    #county_subset=['KING'],
    pev_dist=pev_dist,
    class_dist=class_dist,
    pref_dist=pref_dist,
    home_access_dist=home_access_dist,
    home_power_dist=home_power_dist,
    work_power_dist=work_power_dist,
    res_charging=res_charging,
    work_charging=work_charging,
)

In [ ]:
# save the profiles to a pickle file in case they are needed later (i.e. use it to skip the above cell)
with open(f'./temperature_load_profiles_{gcam_scenario}_{year}_{home_access_dist}_{res_charging}.pickle', 'wb') as handle:
    pickle.dump(temperature_load_profiles, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# skip the long API call above if you have already run it before and saved a pickle file
with open(f'./temperature_load_profiles_{gcam_scenario}_{year}_{home_access_dist}_{res_charging}.pickle', 'rb') as handle:
    temperature_load_profiles = pickle.load(handle)

In [ ]:
# stitch together the temperature load profiles based on the county weather
# you'll need to download and unzip the year of data +/- one day into the specified folder
loads = LDV.get_annual_hourly_load_profiles_by_county(
    fleet_size,
    temperature_load_profiles,
    f'../input/county_meteorology/{weather_scenario}/{year}',
    year,
)

In [ ]:
fips = pd.read_csv('../input/EV_at_scale_2020_Kintner-Meyer_etal.csv')
loads = loads.merge(fips[['State', 'FIPS']], how='left', on='FIPS')

In [ ]:
# sanity check
loads

In [ ]:
# need to scale the EVI-Pro Lite load shapes to match with the GCAM state level output
state_loads = loads.groupby('State')[['load_MWh']].sum()

# read the GCAM transportation energy by state and convert to MW
gcam_transportation_energy = gcam_transportation_energy[gcam_transportation_energy['Year'] == year]
gcam_transportation_energy = gcam_transportation_energy[gcam_transportation_energy['technology'].isin(['BEV', 'Electric'])]
# convert Exajoule to Megawatt
gcam_transportation_energy['mw'] = gcam_transportation_energy['value'] * 277.77777777778 * 1000000 / 8760
ldv_state_level_comparison = (gcam_transportation_energy[gcam_transportation_energy['subsector'].isin([
    '2W and 3W',
    'Car',
    'Large Car and Truck',
    'Light truck',
]) & gcam_transportation_energy.region.isin([
    'CA','OR','WA','AZ','NV','WY','ID','UT','NM','CO','MT'
])].groupby('region')[['mw']].sum() * 8760).merge(
    state_loads,
    left_index=True,
    right_index=True,
).rename(columns={'mw': 'GCAM_MWh', 'load_MWh': 'transportation_MWh'})

ldv_state_level_comparison['scale_factor'] = ldv_state_level_comparison['GCAM_MWh'] / ldv_state_level_comparison['transportation_MWh']

In [ ]:
# sanity check
ldv_state_level_comparison

In [ ]:
# merge the loads with the scale factors
loads = loads.merge(
    ldv_state_level_comparison[['scale_factor']],
    left_on='State',
    right_index=True,
)

In [ ]:
# scale the loads
loads['load_MWh'] = loads['load_MWh'] * loads['scale_factor']

In [ ]:
# aggregate the county loads to the balancing authority level
ba_loads = LDV.aggregate_to_balancing_authority(loads)

In [ ]:
ba_loads

In [ ]:
# sanity check total WECC LDV energy
ba_loads.load_MWh.sum(), (gcam_transportation_energy[gcam_transportation_energy['subsector'].isin([
    '2W and 3W',
    'Car',
    'Large Car and Truck',
    'Light truck',
]) & gcam_transportation_energy.region.isin([
    'CA','OR','WA','AZ','NV','WY','ID','UT','NM','CO','MT'
])].mw.sum())*8760

In [ ]:
# write out csv files per balancing authority
for group, data in ba_loads.groupby('balancing_authority'):
    data.to_csv(f"./output/{gcam_scenario}/{weather_scenario}/{res_charging}/{group}_hourly_LDV_load_{gcam_scenario}_{weather_scenario}_{year}_{home_access_dist}_{res_charging}.csv", index=False)

In [ ]:
# sanity checks

In [ ]:
figs = LDV.plot_county_loads(loads, 53033, show=False)
figs[0].axes[0].set_title(f'LDV load for KING (53033) County for SCL Balancing Authority, {gcam_scenario} {weather_scenario} {res_charging}')

In [ ]:
f = LDV.plot_balancing_authority_loads(ba_loads, 'IID', False)

In [ ]:
f = LDV.plot_balancing_authority_loads(ba_loads, 'AZPS', False)

In [ ]:
f = LDV.plot_balancing_authority_loads(ba_loads, 'CISO', False)

In [ ]:
ciso = pd.read_csv('./output/BAU/rcp45cooler/CISO_hourly_LDV_load_BAU_rcp45cooler_2035.csv', parse_dates=['time'])

In [ ]:
f = LDV.plot_balancing_authority_loads(ciso, 'CISO', False)

<br/>

<br/>